# **Feed Forward Neural Network** (FFNN) Implementation
#### _with **Forward Propagation**_

#### Made by:
- Samuel Christoper Swandi - 13520075
- Grace Claudia - 13520078
- Ubaidillah Ariq Prathama - 13520085
- Patrick Amadeus Irawan - 13520109

------------

List of Content

1. [Library & Dependencies](#library)
2. [Helper Function](#helper)
3. [Neural Network Visualization](#visualization)

## Library & Dependencies <a name="library"></a>

In [1]:
!python3 -m pip install pyvis==0.3.2
!python3 -m pip install networkx==2.6.3
!python3 -m pip install numpy==1.21.6

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import os

from pyvis.network import Network

## Class

#### Connected Layer

In [3]:
class ConnectedLayer:
    def __init__(self, input_size, output_size, weights):
        self.input_size = input_size
        self.output_size = output_size
        self.weights = weights[1:]
        self.bias = weights[0]

    def forward(self, input):
        self.input = input
        return np.dot(input, self.weights) + self.bias

#### Activation Layer

In [4]:
class ActivationLayer:
    def __init__(self, activation):
        self.activation = activation
    
    def forward(self, input):
        self.input = input
        return self.activation(input)

#### Neural Network

In [5]:
class NeuralNetwork:
    def __init__(self):
        self.layers = []

    def add(self, layer):
        self.layers.append(layer)
    
    def predict(self, input_data):
        samples = len(input_data)
        result = []
        complete_result = []

        for i in range(samples):
            complete_result.append([])
            output = input_data[i][1:]
            for layer in self.layers:
                output = layer.forward(output)
                if isinstance(layer, ActivationLayer):
                    complete_result[i].append(np.round(output, 4))
            result.append(output)

        return result, complete_result


## **Helper** Function <a class="anchor" id="helper"></a>

#### Activation Function

In [6]:
def linear(net):
    return net

def ReLU(net):
    return np.maximum(0,net)

def sigmoid(net):
    return 1/(1+np.exp(-net))

def softmax(net):
    return np.exp(net)/np.sum(np.exp(net))

#### Loss Function

In [7]:
def mse(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

def sse(y_true, y_pred):
    return np.sum(np.square(y_true - y_pred))

## Model Load Section

In [8]:
MODEL_FOLDER = '../test/model/'
INPUT_FOLDER = '../test/input/'
# read file from test folder
def read_file(folder_path, file_name):
    with open(folder_path + file_name, 'r') as file:
        data = [i.rstrip("\n") for i in file.readlines()]
    return data

filename = input('Enter model file name: ')
data = read_file(MODEL_FOLDER, filename)
filename = input('Enter input file name: ')
input_data = read_file(INPUT_FOLDER, filename)

## Loading Weights

In [9]:
idx = 1
weights = []
list_prev_size = []
list_layer_size = []
list_activation = []

for i in range(int(data[0]) - 1):
    prev_size, layer_size, activation = [int(i) for i in data[idx].split()]
    list_prev_size.append(prev_size)
    list_layer_size.append(layer_size)
    list_activation.append(activation)
    
    idx += 1
    weight = []
    for j in range(prev_size + 1):
        weight.append([float(i) for i in data[idx].split()])
        idx += 1
    
    weights.append(weight)

## Predict Output

In [10]:
x_train = []
for i in range(len(input_data)):
    x_train.append([float(i) for i in input_data[i].split()])
x_train = np.array(x_train)

n = int(data[0])
act = {0: linear, 1: ReLU, 2: sigmoid, 3: softmax}

net = NeuralNetwork()

for i in range (n - 1):
    net.add(ConnectedLayer(list_prev_size[i], list_layer_size[i], weights[i]))
    net.add(ActivationLayer(act[list_activation[i]]))
    
out, complete_out = net.predict(x_train)
print(out)
print(complete_out)

[array([1.]), array([1.]), array([1.]), array([1.])]
[[array([ 0. , 15.3]), array([1.])], [array([35.  ,  4.91]), array([1.])], [array([0. , 5.3]), array([1.])], [array([34.11,  0.  ]), array([1.])]]


## Neural Network Visualization

In [11]:
n = int(data[0])
act = {0: "linear", 1: "ReLU", 2: "sigmoid", 3: "softmax"}
idx = 1

XSTEP = 300
YSTEP = 300
SIZE = 10

nodes = []
node_i = 1
value = []
x_val = 0
y_val = 0
x = []
y = []
label = []
color = []
edge = []
title = []

src_idx = 0

for i_layer in range(n - 1):
    n_curr, n_next, act = [int(i) for i in data[idx].split()]

    if i_layer == 0:  # means that this is the first layer, hence construct input
        for i_node in range(n_curr + 1):
            if i_node == 0:  # bias
                color.append("#dd4b39")
                label.append("Input[bias]")
                temp = ""
                for i in range (len(input_data)):
                    temp += str(x_train[i][0])
                    if i < len(input_data) - 1:
                        temp += ", "
                title.append(temp)
            else:
                color.append("#162347")
                label.append("Input[{}]".format(i_node))
                temp = ""
                for i in range (len(input_data)):
                    temp += str(x_train[i][i_node])
                    if i < len(input_data) - 1:
                        temp += ", "
                title.append(temp)
            value.append(SIZE)
            x.append(x_val)
            y.append(y_val)
            y_val += YSTEP
            nodes.append(node_i)
            node_i += 1
        x_val += XSTEP

    y_val = 0
    # always construct the next layer
    for i_node in range(n_next + 1):
        if i_node == 0:
            if i_layer == n - 2:
                continue
            color.append("#dd4b39")
            label.append("HL{}[bias]".format(i_layer + 1))
            temp = ""
            for i in range (len(input_data)):
                temp +=  "1"
                if i < len(input_data) - 1:
                    temp += ", "
            title.append(temp)
        else:
            color.append("#162347")
            if i_layer == n - 2:
                label.append("Output[{}]".format(i_node))
            else:
                label.append("HL{}[{}]".format(i_layer + 1, i_node))
            temp = ""
            for i in range (len(input_data)):
                temp += str(complete_out[i][i_layer][i_node - 1])
                if i < len(input_data) - 1:
                    temp += ", "
            title.append(temp)
        value.append(SIZE)
        x.append(x_val)
        y.append(y_val)
        y_val += YSTEP
        nodes.append(node_i)
        node_i += 1
    x_val += XSTEP

    idx += 1
    for origin in range(n_curr + 1):
        dst_idx = -1
        for w in reversed(data[idx].split()):
            edge.append((nodes[src_idx], nodes[dst_idx], w))
            dst_idx -= 1
        src_idx += 1
        idx += 1


In [12]:
g = Network(notebook=True, cdn_resources="remote")
g.add_nodes(
	nodes,
    title = title,
    value = value,
    x=x,y=y,
    label = label,
    color = color
)

for e in edge:
    g.add_edge(e[0], e[1], title = e[2], color="#162347")

for n in g.nodes:
    n.update({'physics': False})
    
g.show("./tmp/network.html")

./tmp/network.html
